In [1]:
import re
import spacy
from spacy.lang.fr import French

# Initialisation du pipeline NLP
nlp = French()
nlp.add_pipe("sentencizer")

def extract_personal_info(text):
    # Nettoyage du texte
    text = re.sub(r'\s+', ' ', text).strip()
    
    # Extraction email (pour avoir un point d'ancrage)
    email = re.search(r'[\w\.-]+@[\w\.-]+', text)
    email = email.group(0) if email else ""
    
    # Stratégie 1: Recherche par motif près de l'email
    name_candidate = ""
    if email:
        # Cherche 2 lignes avant l'email
        context = text[:text.find(email)].strip().split('\n')[-3:]
        for line in context:
            line = line.strip()
            if line and not any(x in line.lower() for x in ['tel', 'phone', '@', 'http']):
                name_candidate = line
                break
    
    # Stratégie 2: Détection NER si stratégie 1 échoue
    first_name, last_name = "", ""
    if not name_candidate:
        doc = nlp(text)
        for ent in doc.ents:
            if ent.label_ == "PER":
                name_parts = ent.text.split()
                if len(name_parts) >= 2:
                    first_name = name_parts[0]
                    last_name = " ".join(name_parts[1:])
                    break
    
    # Stratégie 3: Recherche de motifs typiques
    if not name_candidate and not (first_name and last_name):
        name_match = re.search(r'(?:nom|name)\s*[:=]\s*([^\n]+)', text, re.IGNORECASE)
        if name_match:
            name_candidate = name_match.group(1).strip()
    
    # Traitement du nom candidat
    if name_candidate and not (first_name and last_name):
        name_parts = name_candidate.split()
        if len(name_parts) >= 2:
            first_name = name_parts[0]
            last_name = " ".join(name_parts[1:])
        elif name_parts:
            last_name = name_parts[0]
    
    # Nettoyage final
    first_name = first_name.strip() if first_name else ""
    last_name = last_name.strip() if last_name else ""
    
    return {
        "first_name": first_name,
        "last_name": last_name,
        "email": email
    }

In [3]:
cv_text = """
CURRICULUM VITAE

Prenom: Jean-Michel 
Nom: Dupont
Tél: 06 12 34 56 78
Email: jm.dupont@email.com

FORMATIONS:
- PhD Informatique
"""
print(extract_personal_info(cv_text))

{'first_name': 'CURRICULUM', 'last_name': 'VITAE Prenom: Jean-Michel Nom: Dupont Tél: 06 12 34 56 78 Email:', 'email': 'jm.dupont@email.com'}


In [1]:
import PyPDF2

def extraire_nom_prenom(chemin_pdf):
    """
    Extrait le nom et le prénom d'un fichier PDF.

    Args:
        chemin_pdf (str): Le chemin d'accès au fichier PDF.

    Returns:
        tuple: Un tuple contenant le nom et le prénom, ou None si non trouvés.
    """
    try:
        with open(chemin_pdf, 'rb') as fichier_pdf:
            lecteur_pdf = PyPDF2.PdfReader(fichier_pdf)
            texte_total = ""
            for page in lecteur_pdf.pages:
                texte_total += page.extract_text()

            # Recherche du nom et du prénom (exemple : "Nom: John Doe")
            lignes = texte_total.splitlines()
            nom = None
            prenom = None
            for i, ligne in enumerate(lignes):
                if "Nom:" in ligne:
                    try:
                        nom = lignes[i].split(":")[1].strip()
                    except IndexError:
                        pass
                if "Prénom:" in ligne:
                    try:
                        prenom = lignes[i].split(":")[1].strip()
                    except IndexError:
                        pass
                if nom and prenom:
                    break
                
                #Essai avec autre format d'extraction
                if "Nom et Prénom:" in ligne:
                    try:
                         n_p = lignes[i].split(":")[1].strip().split(" ")
                         nom = n_p[1]
                         prenom = n_p[0]
                    except IndexError:
                        pass
                
                if nom and prenom:
                    break

            if nom and prenom:
                return nom, prenom
            else:
                return None
    except FileNotFoundError:
        print(f"Erreur : Le fichier PDF n'a pas été trouvé à l'emplacement {chemin_pdf}")
        return None
    except Exception as e:
        print(f"Erreur lors de l'extraction : {e}")
        return None

# Exemple d'utilisation:
chemin_fichier = "chemin/vers/votre/fichier.pdf" # Remplacez par le chemin de votre PDF
resultat = extraire_nom_prenom(chemin_fichier)

if resultat:
    nom, prenom = resultat
    print(f"Nom : {nom}, Prénom : {prenom}")
else:
    print("Nom et prénom non trouvés dans le PDF.")

Erreur : Le fichier PDF n'a pas été trouvé à l'emplacement chemin/vers/votre/fichier.pdf
Nom et prénom non trouvés dans le PDF.


In [ ]:
import re

def extract_personal_info(text):
    # Normalisation du texte
    text = re.sub(r'\s+', ' ', text).strip()
    
    # 1. Extraction de l'email (point d'ancrage fiable)
    email_match = re.search(r'[\w\.-]+@[\w\.-]+', text)
    email = email_match.group(0) if email_match else ""
    
    # 2. Stratégies d'extraction du nom
    name = extract_name_from_context(text, email)
    
    # 3. Nettoyage et validation
    return validate_and_format_name(name, email)

def extract_name_from_context(text, email):
    """Multi-stratégies pour trouver le nom"""
    # Stratégie 1: Lignes avant l'email (80% des CV)
    if email:
        context = text[:text.find(email)].strip().split('\n')
        for line in reversed(context[-3:]):  # Regarde les 3 dernières lignes
            line = clean_line(line)
            if is_name_candidate(line):
                return line
    
    # Stratégie 2: Motifs typiques (Nom:, NAME, etc.)
    for pattern in [r'(?:nom|name)\s*[:=]\s*([^\n]+)', 
                   r'^[A-Z][a-z]+\s+[A-Z][a-z]+(?:\s+[A-Z][a-z]+)*$']:
        match = re.search(pattern, text, re.IGNORECASE|re.MULTILINE)
        if match:
            return match.group(1) if ':' in pattern else match.group(0)
    
    # Stratégie 3: Première ligne non-métadonnée
    for line in text.split('\n'):
        line = clean_line(line)
        if is_name_candidate(line):
            return line
    
    return ""

def clean_line(line):
    """Nettoyage des lignes"""
    line = line.strip()
    # Retire les numéros de téléphone
    line = re.sub(r'[\+\d][\d\s\.\-\(\)]{8,}\d', '', line)
    # Retire les emails/URLs
    line = re.sub(r'\S+@\S+|\w+\.\w{2,3}(?:\.\w{2})?', '', line)
    return line.strip()

def is_name_candidate(line):
    """Validation des candidats noms"""
    if not line or len(line.split()) > 4:
        return False
    # Doit contenir au moins 2 mots avec initiales majuscules
    words = line.split()
    return (len(words) >= 1 and 
            all(re.match(r'^[A-ZÀ-Ü][a-zà-ü\-]*$', w) for w in words[:2]))

def validate_and_format_name(name, email):
    """Formattage final"""
    # Extraction prénom/nom depuis l'email si possible
    email_name = ""
    if email and '@' in email:
        email_name = email.split('@')[0]
        email_name = re.sub(r'[\._-]', ' ', email_name).title()
    
    # Fusion des sources
    if not name and email_name:
        parts = email_name.split()
        return {
            "first_name": " ".join(parts[:-1]) if len(parts) > 1 else "",
            "last_name": parts[-1],
            "email": email
        }
    
    # Découpage standard
    parts = name.split()
    if len(parts) >= 2:
        return {
            "first_name": " ".join(parts[:-1]),
            "last_name": parts[-1],
            "email": email
        }
    return {
        "Prenom": "",
        "Nom": name if name else "",
        "email": email
    }

In [4]:
cv_text = """
CURRICULUM VITAE

Prenom: Marie-Claire 
Nom : Dupont
Téléphone: +33 6 12 34 56 78
Email: marieclaire.dupont@email.com

FORMATIONS:
- PhD en Informatique
"""

print(extract_personal_info(cv_text))
# Output: {'first_name': 'Marie Claire', 'last_name': 'Dupont', 'email': 'marieclaire.dupont@email.com'}

{'first_name': 'Marie-Claire Nom : Dupont Téléphone: +33 6 12 34 56 78 Email: marieclaire.dupont@email.com FORMATIONS: - PhD en', 'last_name': 'Informatique', 'email': 'marieclaire.dupont@email.com'}


In [19]:
import requests
from flask import current_app  # Remplacez l'import de Config
import json

first_name = "John JV"
last_name = "McMurray"

api_key = 'aa2a5280646ebe83f26a7f57e4074147'

class ScopusAPI:
    BASE_URL = "https://api.elsevier.com/content"
    
    def __init__(self, api_key=None):
        self.api_key = api_key or current_app.config['SCOPUS_API_KEY']
        self.headers = {
            "Accept": "application/json",
            "X-ELS-APIKey": self.api_key
        }
    
    def search_author(self, first_name, last_name):
        url = f"{self.BASE_URL}/search/author"
        params = {
            "query": f"authfirst({first_name}) and authlast({last_name})",
            "count": 10
        }
        response = requests.get(url, headers=self.headers, params=params)
        return response.json()
    
    def get_author_publications(self, author_id):
        url = f"{self.BASE_URL}/search/scopus"
        params = {
            "query": f"AU-ID({author_id})",
            "view": "STANDARD"
        }
        response = requests.get(url, headers=self.headers, params=params)
        return response.json()
    
    def get_journal_metrics(self, issn):
        url = f"{self.BASE_URL}/serial/title/issn/{issn}"
        params = {
            "view": "METRICS"
        }
        response = requests.get(url, headers=self.headers, params=params)
        return response.json()
    
scopus = ScopusAPI(api_key)
    
result = scopus.search_author("John", "McMurray")
print(json.dumps(result, indent=2))


{
  "service-error": {
    "status": {
      "statusCode": "AUTHORIZATION_ERROR",
      "statusText": "The requestor is not authorized to access the requested view or fields of the resource"
    }
  }
}


In [13]:
entries = result.get("search-results", {}).get("entry", [])
if entries:
    author_id = entries[0]["dc:identifier"].split(":")[-1]
    print("Author ID:", author_id)
else:
    print("Auteur non trouvé.")


Auteur non trouvé.


In [28]:
import requests
import json
from typing import List, Dict

ORCID_API_URL = "https://pub.orcid.org/v3.0"
HEADERS = {
    "Accept": "application/json"
}

from typing import List, Dict
import requests

ORCID_API_URL = "https://pub.orcid.org/v3.0"  # Exemple d'URL d'API ORCID publique
HEADERS = {
    "Accept": "application/json"
}

def get_orcid_publications(orcid_id: str) -> List[Dict]:
    """
    Récupère les publications d'un chercheur via son ID ORCID avec une gestion d'erreur complète
    """
    url = f"{ORCID_API_URL}/{orcid_id}/works"
    try:
        response = requests.get(url, headers=HEADERS, timeout=10)
        response.raise_for_status()
        works = response.json().get('group', [])
    except Exception as e:
        print(f"Erreur lors de la récupération ORCID: {str(e)}")
        return []

    publications = []

    for work in works:
        try:
            work_summary = work.get('work-summary', [{}])[0]
            if not work_summary:
                continue

            # Titre
            title = (work_summary.get('title', {})
                                  .get('title', {})
                                  .get('value', "Titre non disponible"))
            
            # Journal
            journal = (work_summary.get('journal-title', {}).get('value') or
                       work_summary.get('container-name', {}).get('value') or
                       "")

            # Date de publication (année)
            pub_date = work_summary.get('publication-date', {})
            year = (pub_date.get('year', {}).get('value') or
                    (pub_date.get('date-parts', [[None]])[0][0]) or
                    "")

            # DOI
            external_ids = work_summary.get('external-ids', {}).get('external-id', [])
            doi = ""
            for ext_id in external_ids:
                if ext_id.get('external-id-type', '').lower() == 'doi':
                    doi = ext_id.get('external-id-value', '')
                    break

            publications.append({
                'title': str(title),
                'journal': str(journal),
                'year': str(year),
                'doi': doi,
                'type': work_summary.get('type', ''),
                'url': work_summary.get('url', {}).get('value', '')
            })

        except Exception as e:
            print(f"Erreur traitement publication: {str(e)}")
            continue

    return publications


In [29]:
def search_orcid_by_name(family_name: str, given_name: str = "") -> List[Dict]:
    """
    Recherche un ID ORCID à partir d'un nom
    Args:
        family_name (str): Nom de famille
        given_name (str): Prénom (optionnel)
    Returns:
        List[Dict]: Liste des profils correspondants
    """
    url = f"{ORCID_API_URL}/expanded-search/?q=family-name:{family_name}"
    if given_name:
        url += f"+AND+given-names:{given_name}"
    
    response = requests.get(url, headers=HEADERS)
    
    if response.status_code != 200:
        raise ValueError(f"Erreur ORCID API: {response.status_code}")
    
    return response.json().get('expanded-result', [])

In [30]:
def get_publications_from_cv(text: str) -> List[Dict]:
    """
    Workflow complet d'extraction des publications depuis un texte de CV
    """
    # 1. Extraire le nom (utilisez votre fonction existante)
    personal_info = extract_personal_info(text)
    
    # 2. Rechercher l'ORCID correspondant
    orcid_profiles = search_orcid_by_name(
        family_name=personal_info['last_name'],
        given_name=personal_info['first_name']
    )
    
    if not orcid_profiles:
        return []
    
    # 3. Prendre le premier résultat (ou implémenter un meilleur matching)
    orcid_id = orcid_profiles[0]['orcid-id']
    
    # 4. Récupérer les publications
    return get_orcid_publications(orcid_id)

In [32]:
# Exemple avec un ORCID connu
orcid_id = "0000-0002-1825-0097"  # Exemple: un chercheur en informatique
publications = get_orcid_publications(orcid_id)

for pub in publications[:3]:  # Affiche les 3 premières publications
    print(f"Titre: {pub['title']}")
    print(f"Journal: {pub['journal']}")
    print(f"Année: {pub['year']}")
    print(f"DOI: {pub['doi']}\n")

Erreur traitement publication: 'NoneType' object has no attribute 'get'
Erreur traitement publication: 'NoneType' object has no attribute 'get'
Erreur traitement publication: 'NoneType' object has no attribute 'get'
Erreur traitement publication: 'NoneType' object has no attribute 'get'
Erreur traitement publication: 'NoneType' object has no attribute 'get'
Titre: Bulk and surface plasmons in artificially structured materials
Journal: IEEE Transactions on Plasma Science
Année: 1987
DOI: 10.1109/TPS.1987.4316723



In [33]:
import re

def extract_personal_info(text: str) -> dict:
    """
    Extrait prénom, nom et email en gérant explicitement les champs Nom: et Prénom:
    """
    # On conserve le texte AVEC retours à la ligne pour la recherche ligne par ligne
    lines = text.splitlines()

    nom, prenom, email = "", "", ""

    # Recherche email (simple, sur tout le texte)
    email_match = re.search(r'[\w\.-]+@[\w\.-]+', text)
    if email_match:
        email = email_match.group(0)

    # Recherche explicite des champs Nom: et Prénom:
    for line in lines:
        line = line.strip()
        nom_match = re.match(r'Nom\s*[:=]\s*(.+)', line, re.IGNORECASE)
        prenom_match = re.match(r'Prénom\s*[:=]\s*(.+)', line, re.IGNORECASE)
        if nom_match:
            nom = nom_match.group(1).strip()
        if prenom_match:
            prenom = prenom_match.group(1).strip()

    # Si on a bien nom et prénom explicitement
    if nom and prenom:
        return {
            "first_name": prenom,
            "last_name": nom,
            "email": email
        }

    # Sinon fallback heuristique, ex: prendre le nom complet trouvé avant email
    # (optionnel, tu peux remettre ici ta logique heuristique si besoin)

    return {
        "first_name": "",
        "last_name": "",
        "email": email
    }


In [34]:
texte = """CURRICULUM VITAE 
Nom: John JV 
Prénom: MacMurray 
Email: sophie.martin@univ-example.fr 
FORMATION - PhD en Informatique, Université Paris-Saclay (2015-2018) - Master en Intelligence Artificielle, Sorbonne Université (2013-2015) 
PUBLICATIONS 
1. "Deep Learning for Medical Image Analysis", Journal of Artificial Intelligence in Medicine, 2020 
2. "Transformer Models for NLP Tasks", IEEE Transactions on Pattern Analysis, 2019 
3. "Computer Vision Approaches", Springer Nature Computer Science, 2018 
COMPÉTENCES - Machine Learning - Traitement d'images médicales - NLP 
PROJETS - Projet ANR sur l'IA médicale (2019-2022)"""

result = extract_personal_info(texte)
print(result)


{'first_name': 'MacMurray', 'last_name': 'John JV', 'email': 'sophie.martin@univ-example.fr'}


In [38]:
import re
import spacy
from typing import Tuple, Optional

# Charge le modèle français de spaCy
nlp = spacy.load("fr_core_news_sm")

def extract_name(text: str) -> Tuple[Optional[str], Optional[str]]:
    """
    Extrait le prénom et nom d'un texte de CV en utilisant une approche multi-couches.
    Retourne (prénom, nom)
    """
    # Nettoyage initial
    text = re.sub(r'\s+', ' ', text).strip()
    
    # 1. Recherche par motifs explicites
    explicit_result = find_explicit_names(text)
    if explicit_result[0] and explicit_result[1]:
        return explicit_result
    
    # 2. Approche NER avec spaCy
    ner_result = find_names_with_ner(text)
    if ner_result[0] and ner_result[1]:
        return ner_result
    
    # 3. Heuristique basée sur la structure
    structure_result = find_names_by_structure(text)
    if structure_result[0] or structure_result[1]:
        return structure_result
    
    # 4. Fallback: Extraction depuis l'email
    return extract_name_from_email(text)

def find_explicit_names(text: str) -> Tuple[Optional[str], Optional[str]]:
    """Recherche des motifs explicites comme 'Nom:', 'Prénom:'"""
    nom, prenom = None, None
    
    # Recherche des motifs
    nom_match = re.search(r'(?:nom|name)\s*[:=]\s*([^\n]+)', text, re.IGNORECASE)
    prenom_match = re.search(r'(?:prénom|prenom|first\s*name)\s*[:=]\s*([^\n]+)', text, re.IGNORECASE)
    
    if nom_match:
        nom = nom_match.group(1).strip()
    if prenom_match:
        prenom = prenom_match.group(1).strip()
    
    return (prenom, nom)

def find_names_with_ner(text: str) -> Tuple[Optional[str], Optional[str]]:
    """Utilise le NER de spaCy pour détecter les noms"""
    doc = nlp(text)
    persons = [ent.text for ent in doc.ents if ent.label_ == "PER"]
    
    if not persons:
        return (None, None)
    
    # Prend le premier nom détecté et le split
    name_parts = persons[0].split()
    
    if len(name_parts) == 1:
        return (None, name_parts[0])
    elif len(name_parts) == 2:
        return (name_parts[0], name_parts[1])
    else:
        # Pour les noms composés: prénom = tous sauf dernier, nom = dernier
        return (" ".join(name_parts[:-1]), name_parts[-1])

def find_names_by_structure(text: str) -> Tuple[Optional[str], Optional[str]]:
    """Heuristique basée sur la structure typique des CV"""
    lines = [line.strip() for line in text.split('\n') if line.strip()]
    
    # Cherche dans les 5 premières lignes non vides
    for line in lines[:5]:
        # Ignore les lignes avec des emails/numéros
        if '@' in line or re.search(r'[\+\d][\d\s\.\-\(\)]{8,}\d', line):
            continue
        
        # Détection des noms avec majuscules initiales
        if re.match(r'^[A-ZÀ-ÉÈ][a-zà-éè]+(?:\s+[A-ZÀ-ÉÈ][a-zà-éè]+)+$', line):
            parts = line.split()
            if len(parts) == 2:
                return (parts[0], parts[1])
            elif len(parts) > 2:
                return (" ".join(parts[:-1]), parts[-1])
            else:
                return (None, parts[0])
    
    return (None, None)

def extract_name_from_email(text: str) -> Tuple[Optional[str], Optional[str]]:
    """Tente d'extraire le nom depuis l'email (fallback)"""
    email_match = re.search(r'([\w\.-]+)@', text)
    if not email_match:
        return (None, None)
    
    email_prefix = email_match.group(1)
    # Supprime les numéros et séparateurs
    clean_name = re.sub(r'[\d_\-\.]+', ' ', email_prefix).title().strip()
    parts = clean_name.split()
    
    if not parts:
        return (None, None)
    elif len(parts) == 1:
        return (None, parts[0])
    else:
        return (" ".join(parts[:-1]), parts[-1])

# Exemple d'utilisation
if __name__ == "__main__":
    cv_text = """
    CURRICULUM VITAE
    
    Prenom : Jean-Michel 
    Nom : Dupont
    Tél: +33 6 12 34 56 78
    Email: jm.dupont@email.com
    
    FORMATIONS:
    - PhD Informatique
    """
    
    prenom, nom = extract_name(cv_text)
    print(f"Prénom: {prenom}, Nom: {nom}")

Prénom: Jean-Michel Nom : Dupont Tél: +33 6 12 34 56 78 Email: jm.dupont@email.com FORMATIONS: - PhD Informatique, Nom: Jean-Michel Nom : Dupont Tél: +33 6 12 34 56 78 Email: jm.dupont@email.com FORMATIONS: - PhD Informatique


In [47]:
import re
from typing import Tuple, Optional

def extract_explicit_names(text: str) -> Tuple[Optional[str], Optional[str]]:
    """
    Extrait uniquement les noms et prénoms clairement étiquetés dans le texte.
    Retourne (prénom, nom)
    """
    # Dictionnaire des motifs de recherche avec leurs labels
    patterns = {
        'prenom': [
            r'prénom\s*[:=]\s*([^\n]+)',  # Français
            r'prenom\s*[:=]\s*([^\n]+)',  # Variante sans accent
            r'first\s*name\s*[:=]\s*([^\n]+)',  # Anglais
            r'given\s*name\s*[:=]\s*([^\n]+)'  # Alternative
        ],
        'nom': [
            r'nom\s*[:=]\s*([^\n]+)',  # Français
            r'last\s*name\s*[:=]\s*([^\n]+)',  # Anglais
            r'family\s*name\s*[:=]\s*([^\n]+)',  # Alternative
            r'surname\s*[:=]\s*([^\n]+)'  # UK
        ]
    }

    # Normalisation du texte (conserve les retours à la ligne)
    text = text.replace('\r\n', '\n').replace('\r', '\n')
    
    # Recherche des motifs pour chaque champ
    results = {'prenom': None, 'nom': None}
    
    for field in patterns:
        for pattern in patterns[field]:
            match = re.search(pattern, text, re.IGNORECASE)
            if match:
                # Nettoyage du résultat
                value = match.group(1).strip()
                # Suppression des éventuels caractères parasites
                value = re.sub(r'^[^a-zA-ZÀ-ÿ]*|[^a-zA-ZÀ-ÿ]*$', '', value)
                if value:
                    results[field] = value
                    break  # On prend le premier motif trouvé pour chaque champ

    # Gestion des cas où le nom complet est dans un seul champ
    if results['nom'] and ' ' in results['nom'] and not results['prenom']:
        parts = results['nom'].split()
        results['prenom'] = ' '.join(parts[:-1])
        results['nom'] = parts[-1]
    
    return (results['prenom'], results['nom'])

# Exemple d'utilisation
if __name__ == "__main__":
    cv_examples = [
        """Prénom: Jean-Michel
        Nom: Dupont
        Email: jm@example.com""",
        
        """FIRST NAME: John
        LASTNAME: Smith""",
        
        """Personal Information:
        Given Name: Marie
        Family Name: Curie""",
        
        """Informations Personnelles:
        prénom : Pierre
        nom : Martin""",
        
        """SURNAME: Einstein
        GIVEN NAME: Albert"""
    ]

    for cv_text in cv_examples:
        prenom, nom = extract_explicit_names(cv_text)
        print(f"Prénom: {prenom}, Nom: {nom}\n{'-'*40}")

Prénom: Jean-Michel, Nom: Jean-Michel
----------------------------------------
Prénom: John, Nom: Smith
----------------------------------------
Prénom: Marie, Nom: Curie
----------------------------------------
Prénom: Pierre, Nom: Pierre
----------------------------------------
Prénom: Albert, Nom: Einstein
----------------------------------------


In [ ]:
flask shell

SyntaxError: invalid syntax (3700135628.py, line 1)